## Constants

In [1]:
import sys, os
import pandas as pd
import polars as pl
import numpy as np
import subprocess
import gc
import optuna
from datetime import datetime, timezone
import warnings
import xgboost as xgb
import joblib as jl
from sklearn.model_selection import train_test_split
import warnings
from sklearn.metrics import matthews_corrcoef
from mlflow.models import infer_signature
import mlflow
import random
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import StratifiedKFold

today = datetime.now(timezone.utc).strftime("%Y_%m_%d")
warnings.filterwarnings("ignore")

from hyper_params import (
    mushroom_tuning_2024_08_06_1722934727_params,
)

SEED = 108
random.seed(SEED)
N_FOLDS = 10
# model
is_tunning = True
try:
    rs = subprocess.check_output("nvidia-smi")
    device = "cuda" if rs is not None else "cpu"
except (
    Exception
):  # this command not being found can raise quite a few different errors depending on the configuration
    print("No Nvidia GPU in system!")
    device = "cpu"

best_params = {
    "device": device,
    "verbosity": 0,
    "objective": "binary:logistic",
}
best_params.update(mushroom_tuning_2024_08_06_1722934727_params)
best_params

{'device': 'cuda',
 'verbosity': 0,
 'objective': 'binary:logistic',
 'tree_method': 'hist',
 'eta': 0.0696294726051571,
 'max_depth': 0,
 'min_child_weight': 1,
 'gamma': 0.044230646284796976,
 'subsample': 0.9405269471473167,
 'colsample_bytree': 0.2999355523666192,
 'lambda': 0.9746051811186938,
 'alpha': 4.210861941737071}

## Prepare data

In [2]:
y_train_pkl = jl.load("../y_train.pkl")
X_train_pkl = jl.load("../X_train.pkl")

print(f"train size: {X_train_pkl.shape}")

train size: (3116945, 294)


## CV

In [3]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


clf: xgb.XGBClassifier = xgb.XGBClassifier(
    **best_params,
    n_estimators=4000,
    early_stopping_rounds=50,
    enable_categorical=True,
)

In [8]:
from tqdm import tqdm

gc.collect()
skf = StratifiedKFold(n_splits=5)

y_preds = []
y_trues = []
for train_index, test_index in tqdm(skf.split(X_train_pkl, y_train_pkl)):
    X_train, X_test = X_train_pkl[train_index], X_train_pkl[test_index]
    y_train, y_test = y_train_pkl[train_index], y_train_pkl[test_index]

    clf.fit(X=X_train, y=y_train, eval_set=[(X_test, y_test)])

    y_pred = clf.predict(X_test)
    y_preds.append(y_pred)
    y_trues.append(y_test)

    del X_train, X_test, y_train, y_test, y_pred
    gc.collect()
# Concatenate the predictions and true labels
y_preds_concat = np.concatenate(y_preds)
y_trues_concat = np.concatenate(y_trues)
mcc = matthews_corrcoef(y_trues_concat, y_preds_concat)
jl.dump(clf, "../clf.pkl")

0it [00:00, ?it/s]

[0]	validation_0-logloss:0.62820
[1]	validation_0-logloss:0.57473
[2]	validation_0-logloss:0.52958
[3]	validation_0-logloss:0.49820
[4]	validation_0-logloss:0.46109
[5]	validation_0-logloss:0.42675
[6]	validation_0-logloss:0.40349
[7]	validation_0-logloss:0.37546
[8]	validation_0-logloss:0.34886
[9]	validation_0-logloss:0.32811
[10]	validation_0-logloss:0.31119
[11]	validation_0-logloss:0.29010
[12]	validation_0-logloss:0.27295
[13]	validation_0-logloss:0.25676
[14]	validation_0-logloss:0.24259
[15]	validation_0-logloss:0.22939
[16]	validation_0-logloss:0.21778
[17]	validation_0-logloss:0.20767
[18]	validation_0-logloss:0.19754
[19]	validation_0-logloss:0.18727
[20]	validation_0-logloss:0.17760
[21]	validation_0-logloss:0.16727
[22]	validation_0-logloss:0.15871
[23]	validation_0-logloss:0.15267
[24]	validation_0-logloss:0.14421
[25]	validation_0-logloss:0.13781
[26]	validation_0-logloss:0.13341
[27]	validation_0-logloss:0.12675
[28]	validation_0-logloss:0.12048
[29]	validation_0-loglos

1it [00:59, 59.81s/it]

[0]	validation_0-logloss:0.63174
[1]	validation_0-logloss:0.57776
[2]	validation_0-logloss:0.53228
[3]	validation_0-logloss:0.50069
[4]	validation_0-logloss:0.46096
[5]	validation_0-logloss:0.42658
[6]	validation_0-logloss:0.40330
[7]	validation_0-logloss:0.37526
[8]	validation_0-logloss:0.34872
[9]	validation_0-logloss:0.32792
[10]	validation_0-logloss:0.31100
[11]	validation_0-logloss:0.28994
[12]	validation_0-logloss:0.27286
[13]	validation_0-logloss:0.25670
[14]	validation_0-logloss:0.24254
[15]	validation_0-logloss:0.22932
[16]	validation_0-logloss:0.21772
[17]	validation_0-logloss:0.20759
[18]	validation_0-logloss:0.19744
[19]	validation_0-logloss:0.18717
[20]	validation_0-logloss:0.17749
[21]	validation_0-logloss:0.16718
[22]	validation_0-logloss:0.15862
[23]	validation_0-logloss:0.15257
[24]	validation_0-logloss:0.14414
[25]	validation_0-logloss:0.13775
[26]	validation_0-logloss:0.13334
[27]	validation_0-logloss:0.12667
[28]	validation_0-logloss:0.12044
[29]	validation_0-loglos

2it [01:59, 59.46s/it]

...

[0]	validation_0-logloss:0.62826
[1]	validation_0-logloss:0.57482
[2]	validation_0-logloss:0.52971
[3]	validation_0-logloss:0.49843
[4]	validation_0-logloss:0.45900
[5]	validation_0-logloss:0.42487
[6]	validation_0-logloss:0.40173
[7]	validation_0-logloss:0.37388
[8]	validation_0-logloss:0.34748
[9]	validation_0-logloss:0.32675
[10]	validation_0-logloss:0.30990
[11]	validation_0-logloss:0.28894
[12]	validation_0-logloss:0.27191
[13]	validation_0-logloss:0.25683
[14]	validation_0-logloss:0.24267
[15]	validation_0-logloss:0.22946
[16]	validation_0-logloss:0.21786
[17]	validation_0-logloss:0.20773
[18]	validation_0-logloss:0.19760
[19]	validation_0-logloss:0.18735
[20]	validation_0-logloss:0.17768
[21]	validation_0-logloss:0.16736
[22]	validation_0-logloss:0.15877
[23]	validation_0-logloss:0.15272
[24]	validation_0-logloss:0.14427
[25]	validation_0-logloss:0.13788
[26]	validation_0-logloss:0.13347
[27]	validation_0-logloss:0.12681
[28]	validation_0-logloss:0.12057
[29]	validation_0-loglos

4it [03:54, 58.34s/it]

[0]	validation_0-logloss:0.62822
[1]	validation_0-logloss:0.57470
[2]	validation_0-logloss:0.52952
[3]	validation_0-logloss:0.49816
[4]	validation_0-logloss:0.45872
[5]	validation_0-logloss:0.42459
[6]	validation_0-logloss:0.40145
[7]	validation_0-logloss:0.37356
[8]	validation_0-logloss:0.34716
[9]	validation_0-logloss:0.32646
[10]	validation_0-logloss:0.30962
[11]	validation_0-logloss:0.28867
[12]	validation_0-logloss:0.27221
[13]	validation_0-logloss:0.25607
[14]	validation_0-logloss:0.24193
[15]	validation_0-logloss:0.22874
[16]	validation_0-logloss:0.21715
[17]	validation_0-logloss:0.20702
[18]	validation_0-logloss:0.19690
[19]	validation_0-logloss:0.18666
[20]	validation_0-logloss:0.17700
[21]	validation_0-logloss:0.16668
[22]	validation_0-logloss:0.15815
[23]	validation_0-logloss:0.15209
[24]	validation_0-logloss:0.14365
[25]	validation_0-logloss:0.13725
[26]	validation_0-logloss:0.13285
[27]	validation_0-logloss:0.12618
[28]	validation_0-logloss:0.11996
[29]	validation_0-loglos

5it [04:49, 57.99s/it]


['../clf.pkl']

In [9]:
print(f"Validation mcc score: {mcc}")

Validation mcc score: 0.9847779417592476


In [10]:
import joblib as jl

submit_df = jl.load("../submit_df.pkl")
X_test_pkl = jl.load("../X_test.pkl")
lb = jl.load("../lb.pkl")

In [11]:
y_preds = clf.predict(X_test_pkl)
pred_classes = lb.inverse_transform(y_preds)
submit_df["class"] = pred_classes
submit_df.to_csv("../submission.csv", index=False)